In [1]:
import os

In [2]:
%pwd

'c:\\Users\\tusha\\OneDrive\\Desktop\\Data-Science-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\tusha\\OneDrive\\Desktop\\Data-Science-Project'

In [52]:
import pandas as pd

data = pd.read_csv("artifacts/data_ingestion/Food_Time_Data_Set.csv")
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,temperature,humidity,precipitation,weather_description,Unnamed: 14,Traffic_Level,Distance (km),TARGET
0,4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,17.11,77.0,0.0,haze,NaN,Low,NaN,21.66666667
1,B379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,19.50,93.0,0.0,mist,NaN,Very High,37.17,85.26666667
2,5D6D,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,20.45,91.0,0.0,mist,NaN,Low,3.34,28.58333333
3,7A6A,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,23.86,78.0,0.0,mist,NaN,Moderate,10.05,35.18333333
4,70A2,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,26.55,87.0,0.0,mist,NaN,High,9.89,43.45


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           10000 non-null  object 
 1   Delivery_person_ID           10000 non-null  object 
 2   Delivery_person_Age          10000 non-null  float64
 3   Delivery_person_Ratings      10000 non-null  float64
 4   Restaurant_latitude          10000 non-null  float64
 5   Restaurant_longitude         10000 non-null  float64
 6   Delivery_location_latitude   10000 non-null  float64
 7   Delivery_location_longitude  10000 non-null  float64
 8   Type_of_order                10000 non-null  object 
 9   Type_of_vehicle              10000 non-null  object 
 10  temperature                  9995 non-null   float64
 11  humidity                     9995 non-null   float64
 12  precipitation                9995 non-null   float64
 13  weather_descript

In [58]:
data.isnull().sum()

ID                                 1
Delivery_person_ID                 1
Delivery_person_Age                1
Delivery_person_Ratings            1
Restaurant_latitude                1
Restaurant_longitude               1
Delivery_location_latitude         1
Delivery_location_longitude        1
Type_of_order                      1
Type_of_vehicle                    1
temperature                        6
humidity                           6
precipitation                      6
weather_description                6
Unnamed: 14                    10001
Traffic_Level                    916
Distance (km)                    921
TARGET                           542
dtype: int64

In [66]:

data.shape

(10001, 18)

In [79]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_dir: Path
    all_schema: dict

In [80]:
from src.datascience.constant import *
from src.datascience.utils.comman import read_yaml, create_directories

In [81]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config["artifacts_root"]])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config["data_validation"]
        schema = self.schema["COLUMNS"]

        create_directories([config["root_dir"]])

        data_validation_config = DataValidationConfig(
            root_dir=config["root_dir"],
            STATUS_FILE=config["STATUS_FILE"],
            unzip_dir =config["unzip_dir"],
            all_schema=schema
        )
        return data_validation_config


In [82]:
import os
from src.datascience import logger

In [83]:
import pandas as pd

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = True  # Default to True

            data = pd.read_csv(self.config.unzip_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False # No need to continue checking once invalid column found

            with open(self.config.STATUS_FILE, "w") as file:
                file.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            raise e


In [84]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
        raise e

[2025-02-11 19:16:22,900: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2025-02-11 19:16:22,905: INFO: comman: yaml file: params.yaml loaded successfully]
[2025-02-11 19:16:22,911: INFO: comman: yaml file: schema.yaml loaded successfully]
[2025-02-11 19:16:22,916: INFO: comman: created directory at: artifacts]
[2025-02-11 19:16:22,919: INFO: comman: created directory at: artifacts/data_validation]
